<a href="https://colab.research.google.com/github/dkomlen/learning/blob/master/llm/in_context_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers accelerate bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import pipeline

In [4]:
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")


Using device: GPU


In [6]:
# Placeholder model ID. Replace this with a small instruction-tuned model (e.g., TinyLlama, Phi-2, or a 7B instruction variant).
# We use a decoder-only model (the architecture most modern LLMs adopt [9]) optimized for chat/instruction.
MODEL_ID = "HuggingFaceH4/zephyr-7b-beta" # Example of a common instruction-tuned model
# If the above is too large, search for a tiny model like 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

print(f"Loading Model: {MODEL_ID}...")

# Create a text generation pipeline
generator = pipeline(
    "text-generation",
    model=MODEL_ID,
    torch_dtype=torch.bfloat16,
    device=device if device != -1 else "cpu"
)

print("Model Loaded.")

Loading Model: HuggingFaceH4/zephyr-7b-beta...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


Model Loaded.


In [7]:
ZERO_SHOT_TASK = """
You are a text formatter. Convert the following simple phrase into the STYLIZED_FORMAT, where all words are uppercase and separated by underscores.

Phrase: The quick brown fox jumps
STYLIZED_FORMAT:
"""

In [9]:
print("--- Running Zero-Shot Prompt ---")
zero_shot_output = generator(
    ZERO_SHOT_TASK,
    max_new_tokens=20,
    do_sample=False # Use greedy decoding for deterministic output
)[0]['generated_text'] # Access the first item (dictionary) in the list, then the 'generated_text' key.

print(zero_shot_output)
# Note: The model might output something generic, incomplete, or a simple continuation,
# as it has to guess the specific *output format* based only on the natural language description.

--- Running Zero-Shot Prompt ---

You are a text formatter. Convert the following simple phrase into the STYLIZED_FORMAT, where all words are uppercase and separated by underscores.

Phrase: The quick brown fox jumps
STYLIZED_FORMAT:
THE_QUICK_BROWN_FOX_JUMPS

Phr


In [10]:
FEW_SHOT_TASK = """
You are a text formatter. Convert the following simple phrases into the STYLIZED_FORMAT, where all words are uppercase and separated by underscores.

Phrase: machine learning is fun
STYLIZED_FORMAT: MACHINE_LEARNING_IS_FUN

Phrase: attention is all you need
STYLIZED_FORMAT: ATTENTION_IS_ALL_YOU_NEED

Phrase: getting hands dirty with llms
STYLIZED_FORMAT:
"""

In [12]:
print("\n--- Running Few-Shot Prompt ---")
few_shot_output = generator(
    FEW_SHOT_TASK,
    max_new_tokens=20,
    do_sample=False
)[0]['generated_text']

print(few_shot_output)


--- Running Few-Shot Prompt ---

You are a text formatter. Convert the following simple phrases into the STYLIZED_FORMAT, where all words are uppercase and separated by underscores.

Phrase: machine learning is fun
STYLIZED_FORMAT: MACHINE_LEARNING_IS_FUN

Phrase: attention is all you need
STYLIZED_FORMAT: ATTENTION_IS_ALL_YOU_NEED

Phrase: getting hands dirty with llms
STYLIZED_FORMAT:
GETTING_HANDS_DIRTY_WITH_LLMS

Phrase: the
